# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

## Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape, name="input"),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

2024-02-02 21:23:56.878156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-02 21:23:57.049956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-02-02 21:23:57.049979: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-02 21:23:57.050884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized wi

## Train the model

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 6s 13ms/step - loss: 0.3700 - accuracy: 0.8896 - val_loss: 0.0796 - val_accuracy: 0.9790
Epoch 2/15
422/422 [==============================] - 5s 12ms/step - loss: 0.1071 - accuracy: 0.9681 - val_loss: 0.0577 - val_accuracy: 0.9845
Epoch 3/15
422/422 [==============================] - 5s 12ms/step - loss: 0.0795 - accuracy: 0.9759 - val_loss: 0.0457 - val_accuracy: 0.9877
Epoch 4/15
422/422 [==============================] - 5s 12ms/step - loss: 0.0683 - accuracy: 0.9788 - val_loss: 0.0412 - val_accuracy: 0.9898
Epoch 5/15
422/422 [==============================] - 5s 12ms/step - loss: 0.0609 - accuracy: 0.9812 - val_loss: 0.0368 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 5s 12ms/step - loss: 0.0531 - accuracy: 0.9834 - val_loss: 0.0354 - val_accuracy: 0.9903
Epoch 7/15
422/422 [==============================] - 5s 12ms/step - loss: 0.0493 - accuracy: 0.9845 - val_loss: 0.0327 - val_accuracy: 0.9915

## Evaluate the trained model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024829721078276634
Test accuracy: 0.991599977016449


In [22]:
# iterate over the layers and export models from first to current layer to h5 files
for i in range(0, len(model.layers)):
    model_i = keras.Model(inputs=model.inputs, outputs=model.layers[i].output, name=f"model_{i}")
    model_i.save(f"model_{i}/model.h5")
    # print(model_i.summary())

In [7]:
import sys
sys.path.append('..')
from keras2circom.keras2circom import circom, transpiler
circom.dir_parse('../keras2circom/node_modules/circomlib-ml/circuits/', skips=['util.circom', 'circomlib-matrix', 'circomlib', 'crypto'])

In [8]:
for i in range(0, len(model.layers)):
    args = {
        '<model.h5>': f'model_{i}/model.h5',
        '--output': f'model_{i}',
        '--raw': False,
        '--decimals': "18"
    }
    transpiler.transpile(args['<model.h5>'], args['--output'], args['--raw'], args['--decimals'])

In [9]:
import subprocess

commands = []

for i in range(0, len(model.layers)):
    commands.append(['circom', f'model_{i}/circuit.circom', '--r1cs'])

# Run each command sequentially and print their output
for cmd in commands:
    subprocess.call(cmd)

template instances: 11


In [27]:
import tf2onnx

spec = tf.TensorSpec([1, 28, 28, 1], tf.float32, name="input")

for i in range(0, len(model.layers)):
    model_i = keras.models.load_model(f'model_{i}/model.h5')
    tf2onnx.convert.from_keras(
        model_i,
        input_signature=[spec],
        inputs_as_nchw=['input'],
        opset=12,
        output_path=f'model_{i}/model.onnx'
    )

2024-02-02 21:58:15.803473: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.817877: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.863944: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.873370: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.919617: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.932169: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:15.989546: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2024-02-02 21:58:16.045035: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:16.191309: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:16.202005: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2024-02-02 21:58:16.263329: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:16.279377: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:16.356855: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-02-02 21:58:16.370525: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


In [40]:
# Ref: https://github.com/zkonduit/ezkl/blob/bceac2fab530fd01701aec3d8018ce318f6c42e1/examples/notebooks/mnist_vae.ipynb
!RUST_LOG=trace

import os
import ezkl
import json


for i in range(0, len(model.layers)):
    model_path = os.path.join(f'model_{i}/model.onnx')
    settings_path = os.path.join(f'model_{i}/settings.json')

    res = ezkl.gen_settings(model_path, settings_path)
    assert res == True

    # read the settings from json
    with open(settings_path, 'r') as f:
        settings = json.load(f)
    
    # print the "num_rows" from the settings
    print(f"Model {i} num_rows: {settings['num_rows']}")


Model 0 num_rows: 184656
Model 1 num_rows: 236032
Model 2 num_rows: 1384794
Model 3 num_rows: 1399322
Model 4 num_rows: 1399322
Model 5 num_rows: 1399322
Model 6 num_rows: 1406585
